In [91]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('/content/PHL-Sankey (1).csv')
df.head()

,Provinsi,Bahan Baku,Kelompok Kayu Bulat,Jenis Kayu Olahan,Produk Ekspor,Volume(m3)
0,Kalimantan Selatan,Kayu Alam,Hutan Tanaman Industri,Bare Core,Bangunan Prefabrikasi,196.59
1,Kalimantan Timur,Kayu Alam,Hutan Tanaman Industri,Blockboard,Bangunan Prefabrikasi,0.00
2,Jawa Barat,Kayu Alam,Hutan Tanaman Industri,Blockboard,Bangunan Prefabrikasi,5.29
3,Bali,Kayu Alam,Hutan Tanaman Industri,Blockboard,Bangunan Prefabrikasi,0.00
4,Kalimantan Timur,Kayu Alam,Hutan Tanaman Industri,Blockboard,Bangunan Prefabrikasi,0.00


In [92]:
# Membuat dataframe pertama
df1 = df.groupby(['Bahan Baku', 'Kelompok Kayu Bulat'])['Volume(m3)'].count().reset_index()
df1.columns = ['source', 'target', 'value']
df1['source'] = df1['source'].map({'Kayu Tanaman': 'Kayu Tanaman', 'Kayu Alam': 'Kayu Alam', 'Kayu Perkebunan': 'Kayu Perkebunan', 'Setengah Jadi': 'Setengah Jadi', 'Limbah': 'Limbah'})

# Membuat dataframe kedua
df2 = df.groupby(['Kelompok Kayu Bulat', 'Jenis Kayu Olahan'])['Volume(m3)'].count().reset_index()
df2.columns = ['source', 'target', 'value']
df2['target'] = df2['target'].map({ 'Chipwood' : 'Chipwood', 'Kayu Lapis dan LVL' : 'Chipwood', 'Kayu Gergajian':'Chipwood', 'Panel':'Panel', 'Pulp':'Pulp', 'Blockboard':'Blockboard', 'Veneer': 'Veneer', 'Bare Core':'Bare Core' ,'Wood Pellet':'Wood Pellet' ,'Particle Board':'Particle Board','Kayu Olahan Lainnya':'Kayu Olahan Lainnya','Moulding':'Moulding','Finger Joint Board':'Finger Joint Board' })

# Membuat dataframe ketiga
df3 = df.groupby(['Jenis Kayu Olahan', 'Produk Ekspor'])['Volume(m3)'].count().reset_index()
df3.columns = ['source', 'target', 'value']
df3['target'] = df3['target'].map({'Bangunan Prefarikasi': 'Bangunan Prefarikasi', 'Chipwood' : 'Chipwood', 'Furnitur Kayu' : 'Furnitur Kayu', 'Kerajinan':'Kerajinan', 'Panel':'Panel', 'Pulp':'Pulp', 'Paper':'Paper', 'Veneer': 'Veneer', 'Woodworking':'WoodWorking'})

# 2 dataframe untuk supply demand kayu olah
# Menggabungkan kedua dataframe
links = pd.concat([df1, df2], axis=0)
links

,source,target,value
0,Kayu Alam,Hutan Tanaman Industri,588
1,Kayu Alam,Jenis Kayu Lainnya,124
2,Kayu Alam,Kayu Eboni,93
3,Kayu Alam,Kayu Indah,322
4,Kayu Alam,Kelompok Meranti,277
...,...,...,...
72,Rimba Campuran,Moulding,10
73,Rimba Campuran,Particle Board,18
74,Rimba Campuran,Pulp,51
75,Rimba Campuran,Veneer,282


In [93]:
# Mendapatkan daftar unique source dan target dari dataframe links
unique_source_target = list(pd.unique(links[['source', 'target']].values.ravel('K')))
unique_source_target

['Kayu Alam',
 'Kayu Tanaman',
 'Limbah',
 'Setengah Jadi',
 'Hutan Tanaman Industri',
 'Jenis Kayu Lainnya',
 'Kayu Eboni',
 'Kayu Indah',
 'Kelompok Meranti',
 'Rimba Campuran',
 'Bare Core',
 nan,
 'Blockboard',
 'Chipwood',
 'Finger Joint Board',
 'Moulding',
 'Particle Board',
 'Pulp',
 'Veneer',
 'Wood Pellet']

In [94]:
# Membuat dictionary untuk memetakan nilai source dan target menjadi nilai numerik
mapping_dict = {k: v for v, k in enumerate(unique_source_target)}
mapping_dict

{'Kayu Alam': 0,
 'Kayu Tanaman': 1,
 'Limbah': 2,
 'Setengah Jadi': 3,
 'Hutan Tanaman Industri': 4,
 'Jenis Kayu Lainnya': 5,
 'Kayu Eboni': 6,
 'Kayu Indah': 7,
 'Kelompok Meranti': 8,
 'Rimba Campuran': 9,
 'Bare Core': 10,
 nan: 11,
 'Blockboard': 12,
 'Chipwood': 13,
 'Finger Joint Board': 14,
 'Moulding': 15,
 'Particle Board': 16,
 'Pulp': 17,
 'Veneer': 18,
 'Wood Pellet': 19}

In [95]:
links['source'] = links['source'].map(mapping_dict)
links['target'] = links['target'].map(mapping_dict)
links

,source,target,value
0,0,4,588
1,0,5,124
2,0,6,93
3,0,7,322
4,0,8,277
...,...,...,...
72,9,15,10
73,9,16,18
74,9,17,51
75,9,18,282


In [96]:
# Mengubah dataframe links menjadi dictionary dengan orientasi 'list'
links_dict = links.to_dict(orient='list')
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = unique_source_target,
      color = "blue"
    ),
    link = dict(
      source = links_dict["source"],
      target = links_dict["target"],
      value = links_dict["value"]
  ))])
fig.update_layout(title_text="Alur Proses Pemetaan Supply and Demand", font_size=10)
fig.show()
    

In [97]:
data_bahan_baku = pd.read_csv("PHL-Sankey.csv")
jumlah_row_jenis = data_bahan_baku['Bahan Baku'].value_counts().reset_index()
jumlah_row_jenis.columns = ['Bahan Baku', 'Jumlah']

kayu_tanaman = jumlah_row_jenis[jumlah_row_jenis['Bahan Baku'] == 'Kayu Tanaman']['Jumlah'].values[0]
kayu_alam = jumlah_row_jenis[jumlah_row_jenis['Bahan Baku'] == 'Kayu Alam']['Jumlah'].values[0]
perkebunan = jumlah_row_jenis[jumlah_row_jenis['Bahan Baku'] == 'Kayu Perkebunan']['Jumlah'].values[0]
setengahjadi = jumlah_row_jenis[jumlah_row_jenis['Bahan Baku'] == 'Setengah Jadi']['Jumlah'].values[0]
limbah = jumlah_row_jenis[jumlah_row_jenis['Bahan Baku'] == 'Limbah']['Jumlah'].values[0]

# Membuat DataFrame untuk jenis kayu bulat dan jumlahnya
databaku = {
    'Bahan Baku': ['Kayu Tanaman', 'Kayu Alam', 'Kayu Perkebunan', 'Setengah Jadi', 'Limbah'],
    'Jumlah': [kayu_tanaman, kayu_alam, perkebunan, setengahjadi, limbah]
}

df_bahanbaku = pd.DataFrame(databaku)

# Menampilkan tabel
print(df_bahanbaku)

FileNotFoundError: ignored

In [ ]:
import pandas as pd

data_kayu_bulat = pd.read_csv("PHL-Sankey.csv")
jumlah_row_kelompok = data_kayu_bulat['Kelompok Kayu Bulat'].value_counts().reset_index()
jumlah_row_kelompok.columns = ['Kelompok Kayu Bulat', 'jumlah_row']

# Mencari jumlah row untuk setiap kelompok kayu bulat
kelompok_list = ['Hutan Tanaman Industri', 'Rimba Campuran', 'Kelompok Meranti', 'Jenis Khusus',
                 'Kayu Indah', 'JENIS KHUSUS', 'Jenis Kayu Lainnya', 'Kayu Eboni']

jumlah_list = []
for kelompok in kelompok_list:
    if kelompok in jumlah_row_kelompok['Kelompok Kayu Bulat'].values:
        jumlah = jumlah_row_kelompok[jumlah_row_kelompok['Kelompok Kayu Bulat'] == kelompok]['jumlah_row'].values[0]
    else:
        jumlah = 0
    jumlah_list.append(jumlah)

# Membuat DataFrame untuk kelompok kayu bulat dan jumlahnya
data = {
    'Kelompok Kayu Bulat': kelompok_list,
    'Jumlah': jumlah_list
}

df_kayubulat = pd.DataFrame(data)

# Menampilkan tabel
print(df_kayubulat)
